#Phase 2: Modeling & Hyperparameter Tuning 

In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 96 kB 3.7 MB/s 


In [2]:
%matplotlib inline 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel
import matplotlib.pyplot as plt #graphs 
import tensorflow as tf 
import keras_tuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


**Data Augmentation & Test, Train, Validation Split**

In [3]:
#Split validation set by 20%
#Image generator helps creating image aumentation to increase the amount of data we have
#Will implement various rotations and flips to the images to distort
train_gen = ImageDataGenerator(rescale = 1./255, 
                               validation_split=0.20, 
                               rotation_range=45, 
                               width_shift_range=0.2, 
                               height_shift_range=0.2, 
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale = 1./255)


def get_train_set(train_gen, input_size):
  train_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                         class_mode='categorical',
                                         target_size= input_size,
                                         color_mode = 'grayscale',
                                         batch_size= 32, 
                                         shuffle = True, 
                                         subset ='training')
  return train_set

def get_validation_set(train_gen, input_size):
  validation_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                          target_size= input_size,
                                          color_mode = 'grayscale',
                                          class_mode='categorical',
                                          batch_size= 32, 
                                          shuffle = True, 
                                          subset ='validation')
  return validation_set

def get_test_set(test_gen, input_size):
  test_set = test_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/test",
                                         target_size=input_size, 
                                         color_mode = 'grayscale',
                                         class_mode='categorical',
                                         batch_size= 1,
                                         shuffle = True)
  return test_set

In [4]:
#get train, val, test sets 
train_set = get_train_set(train_gen=train_gen, input_size = (224,224))
val_set = get_validation_set(train_gen=train_gen, input_size = (224,224))
test_set = get_test_set(test_gen=test_gen, input_size = (224,224))

Found 6755 images belonging to 196 classes.
Found 1585 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


##Structural Tuning

In [13]:
input_shape = (224,224,1)
model = tf.keras.applications.InceptionV3(include_top = False, weights= None,input_shape=input_shape)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
#input, conv2d, pooling, flatten, Dense, output
#flatten layer = get vector to put in classifier 
# conv layers followed by max pool
# last conv layer followed by a dropout layer to prevent overfitting 
# flatten, fully connected layers 

class Hypermodel(HyperModel):

  def __init__(self, NN_Type):
    self.NN_Type = NN_Type

  def build(self, hp):
    if (self.NN_Type != 'GoogleNet' and self.NN_Type != 'VGG16' and self.NN_Type != 'ResNet152'):
      raise ValueError("Invalid model type")
    if (self.NN_Type == 'GoogleNet'):
      model = tf.keras.applications.InceptionV3(include_top = False, input_shape=(224, 224, 1), classes = 196)
      #flatten
      output_layer = tf.keras.layers('')
      x = tf.keras.layers.Flatten()
      #dense
      #dropout
      #classification layer(dense)
    elif (self.NN_Type == 'VGG16'):
      tf.keras.applications.VGG16()
    else: 
      tf.keras.applications.ResNet152()
    return model
  
  def summary(self):


    

In [ ]:
#don't explore unlikely options 
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

#create model checkpoint (save at this point)


hypermodel_Google = Hypermodel(NN_Type = 'GoogleNet')
hypermodel_VGG = Hypermodel(NN_Type = 'VGG16')
hypermodel_ResNet = Hypermodel(NN_Type = 'GoogleNet')

tuner_GN = Hyperband(hypermodel =hypermodel_Google, objective = 'val_accuracy', max_epochs = 100)
tuner_VGG = Hyperband(hypermodel=hypermodel_VGG, objective='val_accuracy', max_epochs=100)
tuner_ResNet = Hyperband(hypermodel=hypermodel_ResNet, objective='val_accuracy', max_epochs=100)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


**GoogleNet Validation**

In [ ]:
tuner_GN.search_space_summary()

Search space summary
Default search space size: 4
num_filters (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout (Float)
{'default': 0.005, 'conditions': [], 'min_value': 0.0, 'max_value': 0.1, 'step': 0.01, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001], 'ordered': True}


In [ ]:
tuner_GN.search(train_set, epochs = 50, validation_data = val_set, callbacks = [stop_early])

Trial 90 Complete [00h 02m 38s]
val_accuracy: 0.06813880056142807

Best val_accuracy So Far: 0.10599368810653687
Total elapsed time: 03h 29m 38s
INFO:tensorflow:Oracle triggered exit


**VGG16 Validation**

**ResNet152 Validation**

##Epoch Optimization for Best Models

In [ ]:
# get best hp
best_hps = tuner_3.get_best_hyperparameters(num_trials =1)[0]
model = tuner_3.hypermodel.build(best_hps)

In [ ]:
#num filters, dropout, densely connected layer 
print("Best num_filters: %d" %(best_hps.get('num_filters')))
print("Best dropout: %f" %(best_hps.get('dropout')))
print("Best number of units: %d" % (best_hps.get('units')))

Best num_filters: 32
Best dropout: 0.050000
Best number of units: 448


In [ ]:
hist = model.fit(train_set, epochs = 120, validation_data = val_set)
val_acc_per_epoch = hist.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/120
212/212 [==============================] - 16s 75ms/step - loss: 5.2710 - accuracy: 0.0067 - val_loss: 5.1722 - val_accuracy: 0.0189
Epoch 2/120
212/212 [==============================] - 16s 74ms/step - loss: 4.5798 - accuracy: 0.0836 - val_loss: 4.7165 - val_accuracy: 0.0644
Epoch 3/120
212/212 [==============================] - 17s 79ms/step - loss: 2.4509 - accuracy: 0.4289 - val_loss: 5.4789 - val_accuracy: 0.0751
Epoch 4/120
212/212 [==============================] - 16s 77ms/step - loss: 0.6114 - accuracy: 0.8623 - val_loss: 6.9725 - val_accuracy: 0.0738
Epoch 5/120
212/212 [==============================] - 17s 79ms/step - loss: 0.2465 - accuracy: 0.9553 - val_loss: 7.1976 - val_accuracy: 0.0776
Epoch 6/120
212/212 [==============================] - 16s 77ms/step - loss: 0.1627 - accuracy: 0.9708 - val_loss: 6.8413 - val_accuracy: 0.0858
Epoch 7/120
212/212 [==============================] - 16s 74ms/step - loss: 0.1175 - accuracy: 0.9726 - val_loss: 6.7241 - val_ac

*Experiment Notes: My model is performing well on the training data and non-optimal * 